<a href="https://colab.research.google.com/github/thomasshin/NLP_Study/blob/main/NLP_with_Transformers/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text Generation


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00


##Greedy Search Decoding

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [ ]:
import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration["input"] = tokenizer.decode(input_ids[0])
    output = model(input_ids=input_ids)
    #softmax the logits from the last token of each batch
    next_token_logits = output.logits[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=-1)
    #sort in descending order
    sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
    #load the tokens from highest probabilities
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)"
      iteration[f"Choice {choice_idx + 1}"] = token_choice
    #add the predicted token to output
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
    iterations.append(iteration)
print(pd.DataFrame(iterations))

                                               input           Choice 1  \
0                               Transformers are the       most (9.76%)   
1                          Transformers are the most    common (22.90%)   
2                   Transformers are the most common      type (15.06%)   
3              Transformers are the most common type        of (83.13%)   
4           Transformers are the most common type of   particle (1.55%)   
5  Transformers are the most common type of particle         . (14.26%)   
6  Transformers are the most common type of parti...      They (17.48%)   
7  Transformers are the most common type of parti...       are (38.78%)   

            Choice 2            Choice 3          Choice 4  \
0       same (2.94%)        only (2.87%)      best (2.38%)   
1   powerful (6.88%)   important (6.32%)   popular (3.95%)   
2      types (3.31%)        form (1.91%)       way (1.89%)   
3         in (3.16%)           . (1.92%)         , (1.63%)   
4     object (

using generate() function

In [ ]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(output)
print("-----------------------------")
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[41762,   364,   389,   262,   749,  2219,  2099,   286, 18758,    13,
          1119,   389]], device='cuda:0')
-----------------------------
Transformers are the most common type of particle. They are


let's try OpenAI's unicorn script

In [ ]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very


Downside of greedy method: repeating output sequences

we can use **beam search decoding** instead

#Beam Search Decoding

In [ ]:
#logit normalization
import torch.nn.functional as F

def log_probs_from_logits(logits, labels): #for a single token
  logp = F.log_softmax(logits, dim=-1) #equivalent to log(softmax(x)) doing these two operations separately is slower and numerically unstable.
  logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
  return logp_label

def sequence_logprob(model, labels, input_len=0): #for the whole sequence
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(output.logits[:, :-1, :], labels[:, 1:])
    seq_log_probs = torch.sum(log_probs[:, input_len:])
  return seq_log_probs.cpu().numpy()

In [ ]:
#calculation of log probability

logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))

print(tokenizer.decode(output_greedy[0]))
print(f"log probability: {logp}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very
log probability: -83.325927734375


In [ ]:
#let's see how beam search does

output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp_ = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"log probability: {logp_}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, San Diego, and the University of California, Santa Cruz, found that the unicorns were able to communicate with each other in a way that was similar to that of human speech.


"The unicorns were able to communicate with each other in a way that was similar to that of human speech," said study co-lead author Dr. David J.
log probability: -78.34051513671875


higher logp for beam search (higher means good), but beam search also have the problem of repeating seqs, so we have to add "no_repeat_ngram_size" param

In [ ]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)
logp_ = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"log probability: {logp_}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, San Diego, and the National Science Foundation (NSF) in Boulder, Colorado, were able to translate the words of the unicorn into English, which they then translated into Spanish.

"This is the first time that we have translated a language into an English language," said study co-author and NSF professor of linguistics and evolutionary biology Dr.
log probability: -101.87501525878906


#Sampling method

temperature = variability

In [ ]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0.0) #temperature has to be float value
print(tokenizer.decode(output_temp[0])) #extremely varies

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


Secondly main friends congregated takes birth Falls snakeLocal said eastern beauty commodulatory species beauty boycot goal to growostyles Whip feathers girlfriends looking airsclud126 timing neocibur VimAdlesh RFC GrimmcentLiveskinny Saturn graduating carirmstand unaware Fidel union�ル abnorm tiers deadaveenei MIL Azure and Ostvisionrbakespeare35Password wisdomprom fix flawsMost invoked that diet cogaccascade CSIbootlegged groundwater


In [ ]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.1, top_k=0.0) #temperature has to be float value
print(tokenizer.decode(output_temp[0])) #much more consistent

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"We were surprised to find that the unicorns were able to communicate with each other," said lead author Dr. David L. Smith, a professor of biology at the University of California, San Diego. "We were surprised to find that the unicorns were able to communicate with each other."


The researchers found that the unicorns were able to communicate with each other by using their vocal cords to


we have to find a balance between consistency and variability

two possible solutions, which remove rare words from corpus (cutting the distribution of corpus):

1. top-k sampling
2. nucleus (top-p) sampling

#Top-k sampling

- sampling from top-k tokens

In [ ]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The findings were reported in a study published online April 6 in the open access journal Science.


"A long time ago, we thought that the unicorns could talk, but now we know they don't, and what a leap in speech development does it have for communicating in an untrained population," said co-founder and CEO of Cornell's School of Zoology, Professor Mark Wiles. "


how to choose k? no definite answer.. moreover a fixed cutoff may not be good enuf, so we came up with a dynamic cutoff

#Top-p sampling

- dynamic cutoff with probability mass

In [ ]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"We were expecting more in language than it was," said co-author Thomas M. Dolan, a Cornell University assistant professor in the Department of Human Evolutionary Anthropology. "But we couldn't find any."


The team came to an even more surprising conclusion: the unicorns also spoke perfect English.


According to M. Dolan, the unicorns learned from the first of their
